In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os, sys
import numpy as np
from typing import Union

import matplotlib.pyplot as plt
import seaborn as sb

import ipywidgets as widgets
from IPython.display import display

In [ ]:
from hmt.lib.fins import (
    FinRectangularPiniform,
    FinRectangularPlanar,
    FinTriangularPlanar,
    FinParabolicPlanar,
    FinAnnular
)

In [ ]:
# sb.set_context(
#     "talk",
#     font_scale=1.2
# )

sb.set_style(
    "darkgrid"
)

layout = widgets.Layout(
            width="50%"
         )

style  = {"description_width": "initial"}

In [ ]:
# Physical constants for parametric analysis
kCu = 398.0 # W/mK
kAl = 180.0 # W/mK
kSteel = 14.0 # W/mK

hNaturalConv = 5.0 # W/m2K
hForcedConv  = 1000.0 # W/m2K
hIntermediate = 100.0 

In [ ]:
# Sliders for interactive plots 
thermalCondSlider = widgets.FloatSlider(
    value=kCu,
    min=kSteel,
    max=kCu,
    step=(kCu - kSteel)/100,
    description="k (W/mK)",
    readout_format='.1f',
    layout=layout,
    style=style
)

htcSlider = widgets.FloatSlider(
    value=hIntermediate,
    min=hNaturalConv,
    max=hForcedConv,
    step=(hForcedConv - hNaturalConv)/500,
    description="h (W/m2K)",
    readout_format='.1f',
    layout=layout,
    style=style
)

## Análise do Exemplo 3.9 Item 1 do Livro-texto

Um bastão muito longo, com 5 mm de diâmetro, tem uma de suas extremidades mantida a 100 °C. A superfície do bastão está exposta ao ar ambiente a 25 °C, com um coeficiente de transferência de calor por convecção de 100 $W/(m^2·K)$.

1. Determine as distribuições de temperaturas ao longo de bastões construídos em cobre puro, liga de alumínio 2024 e aço inoxidável AISI 316. Quais são as respectivas taxas de perda de calor nos bastões?

In [ ]:
# Dados do problema
exHtc   = 100.0 # W/m2K
exTbase = 100.0 # Celsius
exTinf  = 25.0 # Celsius

# Dimensions
exFinRadius = 2.5e-3 # meters

Vamos supor um comprimento $L = 300 mm$ para esta aleta para avaliarmos o campo de temperatura na mesma.

In [ ]:
finLength = 0.3 # meters

A implementação feita aqui usa classes em Python para encapsular todas as operações necessárias para estudarmos aletas. Podemos instanciar um objeto da classe "PiniformFin" (aleta piniforme com área de seção uniforme) e informar as grandezas da mesma.

Apenas como exemplo do tipo de interação possível, vamos instanciar um objeto de aleta com a condutividade do cobre. Todas as grandezas devem ser dadas em unidades do SI, com exceção da temperatura que pode ser informada em graus Celsius.

In [ ]:
pinFin = FinRectangularPiniform(
            length=finLength,
            diameter=2*exFinRadius,
            htc=exHtc,
            k=kCu,
            Tinf=exTinf,
            Tbase=exTbase
       )

Agora podemos acessar as grandezas da mesma usados suas chamadas "funções membro", como segue: 

In [ ]:
# Area da superfície
pinFin.getSurfaceArea() 

In [ ]:
# Perímetro
pinFin.getPerimeter() 

In [ ]:
# Area da seção transversal
pinFin.getTrArea()

In [ ]:
# Taxa total de transf. de calor (W)
pinFin.getHeatTransfer()

In [ ]:
# Effetividade
pinFin.getEffectiveness()

In [ ]:
# Efficiency
pinFin.getEfficiency()

Vamos agora à comparação da distribuição de temperaturas para os 3 casos de materias considerados da aleta piniforme. Bastar executar o código a seguir. A efetividade de cada caso é mostrada na legenda da figura.

In [ ]:
nPoints = 100
xRange = np.linspace(
            0.0,
            finLength,
            nPoints
         )

In [ ]:
fig, axes = plt.subplots(
                figsize=(8,8)
            )

for k, label in zip([kCu, kAl, kSteel],
                    ["Cobre", "Alumínio", "Aço"]):
    
    # Instantiate piniform fin model
    pinFin = FinRectangularPiniform(
                 length=finLength,
                 diameter=2*exFinRadius,
                 htc=exHtc,
                 k=k,
                 Tinf=exTinf,
                 Tbase=exTbase
             )

    # Get temperature distribution
    xRange = np.linspace(
                    0.0,
                    pinFin.getLength(),
                    nPoints
                )

    pinTemps = [pinFin.getTemperature(x)
                for x in xRange]
    
    sb.lineplot(
        x=xRange,
        y=pinTemps,
        label=f"{label} {pinFin.getEffectiveness():.2f}",
        ax=axes
    )

    axes.set(
        ylabel="T (ºC)",
        xlabel="x (m)",
        title="Comparação de T(x) para diferentes materiais \n Aleta piniforme",
        ylim=(20,105)
    )

Claramente a efetividade da aleta de aço é menor, o que é esperado. A efetividade da aleta é melhorada quando selecionamos um material com maior condutividade térmica.

Uma versão interativa deste gráfico pode ser encontrado com o código a seguir. Use os "sliders" para controlar o valor:

1. da condutividade térmica;
2. do coef. de transferência de calor por conveccção;
3. e do raio da aleta.

In [ ]:
pinRadiusSlider = widgets.FloatSlider(
    value=exFinRadius,
    min=exFinRadius,
    max=5.0*exFinRadius,
    step=exFinRadius/100,
    readout_format='.2e',
    layout=layout,
    style=style,
    description="R (m)"
)

In [ ]:
@widgets.interact(
    k=thermalCondSlider,
    h=htcSlider,
    R=pinRadiusSlider
)
def InteractiveTempFin(k,h,R):
    
    fig, axes = plt.subplots(
                    # figsize=(8,6)
                )
    
    pinFin = FinRectangularPiniform(
                 length=finLength,
                 diameter=2*R,
                 htc=h,
                 k=k,
                 Tinf=exTinf,
                 Tbase=exTbase
             )

    pinTemps = [pinFin.getTemperature(x)
                for x in xRange]
    
    sb.lineplot(
        x=xRange,
        y=pinTemps,
        ax=axes
    )

    axes.text(
        0.32, 100,
        f"Heat transfer = {pinFin.getHeatTransfer():.2f} W\n"\
        f"Efetividade = {pinFin.getEffectiveness():.2f}\n"\
        f"Eficiência = {pinFin.getEfficiency():.2f}",
        ha = "left",
        va = "top"
    )

    axes.set(
        ylabel="T (ºC)",
        xlabel="x (m)",
        title="Comparação de T(x) para diferentes materiais \n Aleta piniforme",
        ylim=(20,105)
    )

## Desempenhos de Aletas

Note que para um comprimento de aproximadamente $L \approx 60mm$ da aleta de aço a aleta não troca mais calor e o resot de material é inútil em vista de sua aplicação. Este valor pode ser obtido ao se comparar a taxa de trans. de calor total da aleta com o modelo de comprimento infinito e o modelo de fronteira adiabática, pois em uma aleta com um comprimento muito longo não há mais troca de calor na sua extremidade. Assim, com esta comparação, podemos avaliar a partir de qual comprimento o modelo de aleta infinita é válido. Comparando as equações (3.81) e (3.85), podemos escrever:

\begin{equation}
    \tanh{mL} \ge 0.99
\end{equation}

O que leva a (podemos resolver esta equação usando a definição da tangente hiperbólica):

\begin{equation}
    mL \ge 2.65
    \rightarrow
    L \ge \frac{2.65}{m}
\end{equation}

sendo $m = \sqrt{\frac{hP}{kA_{tr}}}$. Assim, na prática, o comprimento das aletas deve ser menor que aproximadamente $\frac{2.65}{m}$ pois a partir desse comprimento ela não troca calor mais. No caso do problema do livro:

In [ ]:
for k, mat in zip([kCu, kAl, kSteel],
                  ["Cobre", "Al", "Aço"]):
    
    fin = FinRectangularPiniform(
                length=finLength,
                diameter=2*exFinRadius,
                htc=exHtc,
                k=k,
                Tinf=exTinf,
                Tbase=exTbase
            )
    
    print(
        f"{mat} -> L >= {2.65/fin._mCoeffFin():.3f} m"
    )

Além disso, note que a efetividade e eficiência da aleta aumetam conforma k aumenta e h diminui. Se aumentarmos o raio da aleta piniforme, aumentamos ainda mais a eficiência mas a efetividade começa a diminuir! Estes efeitos são esperados.

Como vimos em aula, a partir da definição da efetividade e assumindo um modelo de aleta infinita, então podemos mostrar que a efetividade de aletas de seção uniforme é dada por:

\begin{equation}
    \epsilon_a
    =
    \left(
        \frac{
            kP
        }{
            hA_{tr}
        }
    \right)^{\frac{1}{2}}
\end{equation}

Assim, a efetividade:

1. aumenta com um valor maior de k;
2. diminui com um valor maior de h;
3. Aumenta com uma maior valor da razão $\dfrac{P}{A_{tr}}$

No caso de aleta piniforme:

\begin{equation}
    \frac{P}{A_{tr}}
    =
    \frac{
        2\pi R
    }{
        \pi R^2
    }
    =
    \frac{2}{R}
\end{equation}

Logo, quanto menor o raio da aleta piniforme, realmente maior vai ser sua efetividade. 

No caso de aletas planas retangulares, temos o seguinte:

\begin{equation}
    \frac{P}{A_{tr}}
    =
    \frac{
        2(w + t)
    }{
        wt
    }
    =
    \frac{
        2\left(1 + \dfrac{t}{w}\right)
    }{
        t
    }
\end{equation}

Assim, se $w \gg t$, quanto menor for a espessura da aleta melhor, pois sua efetividade aumentará.

In [ ]:
# Adding a similar planar fin
exFinWidth = 0.20
exFinThickness = 0.01

planarFin = FinRectangularPlanar(
                length=finLength,
                width=exFinWidth,
                thickness=exFinThickness,
                htc=exHtc,
                k=kCu,
                Tinf=exTinf,
                Tbase=exTbase
           )

In [ ]:
localRadiusSlider = widgets.FloatSlider(
    value=exFinRadius,
    min=exFinRadius,
    max=5.0*exFinRadius,
    step=exFinRadius/100,
    readout_format='.2e',
    layout=layout,
    style=style
)

@widgets.interact(R=localRadiusSlider)
def InteractiveCompareFinSizes(
        R
    ):

    fig, ax = plt.subplots()
    
    pinFin = FinRectangularPiniform(
                length=finLength,
                diameter=2*R,
                htc=exHtc,
                k=kCu,
                Tinf=exTinf,
                Tbase=exTbase
           )
    
    sb.lineplot(
        x=xRange,
        y=[pinFin.getTemperature(x)
           for x in xRange],
        ax=ax,
        label=f"Piniform (V = {1.0e9*pinFin.getVolume():.2f} mm3)"
    )

    sb.lineplot(
        x=xRange,
        y=[planarFin.getTemperature(x)
           for x in xRange],
        ax=ax,
        label=f"Planar (V = {1.0e9*planarFin.getVolume():.2f} mm3)"
    )
    
    ax.set(
        ylabel="Temperature (ºC)",
        xlabel="x (m)",
        title="What radius a piniform fin must have \n to compare with a planar one?",
        ylim=(exTinf, exTbase)
    )

In [ ]:
@widgets.interact(
    k=thermalCondSlider,
    h=htcSlider
)
def InteractiveTempFin(k,h):
    
    fig, axes = plt.subplots(
                    figsize=(10,8)
                )

    lengths = np.linspace(0,0.50,50)

    planarEfficiencies = [FinRectangularPlanar(
                                 length=L,
                                 width=0.2,
                                 thickness=0.01,
                                 htc=h,
                                 k=k,
                                 Tinf=exTinf,
                                 Tbase=exTbase
                             ).getEfficiency() 
                            for L in lengths]


    pinEfficiencies = [FinRectangularPiniform(
                             length=L,
                             diameter=2.0*exFinRadius,
                             htc=h,
                             k=k,
                             Tinf=exTinf,
                             Tbase=exTbase
                         ).getEfficiency() 
                        for L in lengths]

    # Define annular fin with similar length and thickness
    innerRadius = 25e-3
    annularEfficiencies = [FinAnnular(
                                 inner_radius=innerRadius,
                                 outer_radius=innerRadius + L,
                                 thickness=0.01,
                                 htc=h,
                                 k=k,
                                 Tinf=exTinf,
                                 Tbase=exTbase
                             ).getEfficiency() 
                            for L in lengths]
    sb.lineplot(
        x=lengths,
        y=planarEfficiencies,
        label="Planar",
        ax=axes
    )

    sb.lineplot(
        x=lengths,
        y=pinEfficiencies,
        label="Piniform",
        ax=axes
    )
    
    sb.lineplot(
        x=lengths,
        y=annularEfficiencies,
        label="Annular",
        ax=axes
    )
    
    axes.set(
        title="Effect of h and k on fin efficiency",
        ylabel="Efficiency",
        xlabel="L (m)",
        ylim=(0.0,1.05)
    )

In [ ]:
localRadiusRatioSlider = widgets.FloatSlider(
    value=1,
    min=1,
    max=10,
    step=0.5,
    readout_format='.2f',
    layout=layout,
    style=style
)

@widgets.interact(
    radiusRatio=localRadiusRatioSlider
)
def InteractiveTempFin(
    radiusRatio
):
    lengths = np.linspace(0,0.50,50)
    
    # Define annular fin with similar length and thickness
    innerRadius = 25e-3
    
    annularEfficiencies = []
    profAreas = []
    
    for L in lengths:
        fin = FinAnnular(
                 inner_radius=innerRadius,
                 outer_radius=innerRadius + L,
                 thickness=exFinThickness,
                 htc=exHtc,
                 k=kCu,
                 Tinf=exTinf,
                 Tbase=exTbase
             )
    
        annularEfficiencies.append(
            fin.getEfficiency()
        )
        
        profAreas.append(
            fin.getProfileArea()
        )
    
    lengths = np.array(lengths)
    profAreas = np.array(profAreas)
    
    modLengths = np.power(
                        lengths + 0.5*exFinThickness, 1.5
                    )*np.sqrt(
                        exHtc/(kCu*profAreas)
                    )
    
    fig, axes = plt.subplots(
                # figsize=(10,8)
            )
    
    sb.lineplot(
        x=modLengths,
        y=annularEfficiencies,
        ax=axes
    )
    
    axes.set(
        title="Effect of h and k on fin efficiency",
        ylabel="Efficiency",
        xlabel="L (m)",
        # ylim=(0.0,1.05)
    )